In [1]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_simulated_annealing
from random import shuffle
import csv

In [2]:
filename = "fallout_4_classical_radio.csv"

def try_type(value):

    try:
        return int(value)
    except ValueError:
        try:
            return float(value)
        except ValueError:
            return value

with open(filename, "r") as csvfile:
    reader = csv.reader(csvfile)
    header, *tracks = list(reader)
    tracks = [
        dict(zip(map(lambda h: h.strip(), header), map(try_type, track)))
        for track in tracks
    ]
    
print(reader)

min_tempo, max_tempo = min(map(lambda track: track["Tempo"], tracks)), max(
    map(lambda track: track["Tempo"], tracks)
)

In [3]:
def distance_function(track1, track2):


    same_mode = track1["Mode"] == track2["Mode"]
    key1 = track1["Key"]
    key2 = track2["Key"]

    if same_mode:
        key_diff = min(abs(key1 - key2), 12 - abs(key1 - key2))
        if key_diff == 6:
            key_distance = 0.5
        elif key_diff <= 1:
            key_distance = 0
        elif key_diff <= 2:
            key_distance = 0.5
        else:
            key_distance = 1
    else:
        key_distance = 0 if key1 == key2 else 1


    tempo1 = (track1["Tempo"] - min_tempo) / (max_tempo - min_tempo)
    tempo2 = (track2["Tempo"] - min_tempo) / (max_tempo - min_tempo)
    tempo_distance = abs(tempo1 - tempo2)


    loudness1 = (track1["Loudness"] - -60) / (0 - -60)
    loudness2 = (track2["Loudness"] - -60) / (0 - -60)
    loudness_distance = abs(loudness1 - loudness2)


    energy_distance = abs(track1["Energy"] - track2["Energy"])
    valence_distance = abs(track1["Valence"] - track2["Valence"])
    danceability_distance = abs(track1["Danceability"] - track2["Danceability"])

    weights = {
        "key_weight": 3,
        "tempo_weight": 0,
        "loudness_weight": 0,
        "energy_weight": 3,
        "valence_weight": 44,
        "danceability_weight": 2,
    }

    song_distance = sum(
        map(
            lambda x: weights[x[0]] * x[1],
            zip(
                weights.keys(),
                [
                    key_distance,
                    tempo_distance,
                    loudness_distance,
                    energy_distance,
                    valence_distance,
                    danceability_distance,
                ],
            ),
        )
    )
    globals().update(locals())
    return song_distance

In [4]:

distances = np.array(
    [[distance_function(track1, track2) for track2 in tracks] for track1 in tracks]
)
distances[:, 0] = 0 
x0 = list(range(len(tracks)))
shuffle(x0)




In [5]:
permutation, distance = solve_tsp_simulated_annealing(distances, x0=x0, alpha=0.999)

with open("sorted_tracks.csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(
        [
            "Track",
            "Old_order",
            "New_order",
            "Key",
            "Mode",
            "Energy",
            "Valence",
            "Danceability",
            "Tempo",
            "Loudness",
        ]
    )
    for new_pos, i in enumerate(permutation):
        track = tracks[i]
        writer.writerow(
            [
                track["Track Name"],
                tracks.index(tracks[i]) + 1,
                new_pos + 1,
                track["Key"],
                track["Mode"],
                track["Energy"],
                track["Valence"],
                track["Danceability"],
                track["Tempo"],
                track["Loudness"],
            ]
        )

In [6]:
for i, j in zip(permutation, permutation[1:]):
    print(tracks[i]["Track Name"], "-->", tracks[j]["Track Name"])

normalized_distance = distance / sum(weights.values())
avg_distance_per_track = normalized_distance / len(tracks)

print(f"norm_dist: {normalized_distance}\navg_dist_per_track: {avg_distance_per_track}")


Die Walkure, Act III: Ride of the Valkyries --> The Planets, Op. 32: I. Mars - The Bringer Of War
The Planets, Op. 32: I. Mars - The Bringer Of War --> Peer Gynt Suite No. 1, Op. 46 : IV. In the Hall of the Mountain King
Peer Gynt Suite No. 1, Op. 46 : IV. In the Hall of the Mountain King --> Nocturne, Op. 9 No. 2
Nocturne, Op. 9 No. 2 --> Saint-Saëns: Le carnaval des animaux, R 125: IX. Le coucou au fond des bois
Saint-Saëns: Le carnaval des animaux, R 125: IX. Le coucou au fond des bois --> Peer Gynt-Suite Nr. 1, Op. 46: I. Morgenstemning (Morning Mood)
Peer Gynt-Suite Nr. 1, Op. 46: I. Morgenstemning (Morning Mood) --> La Mer, L.109: 2. Play Of The Waves (Jeux de vagues)
La Mer, L.109: 2. Play Of The Waves (Jeux de vagues) --> Variation IX (Nimrod): Adagio
Variation IX (Nimrod): Adagio --> Khovanshchina: Prelude
Khovanshchina: Prelude --> Scheherazade: I. The Sea & Sinbad's Ship
Scheherazade: I. The Sea & Sinbad's Ship --> Nocturnes : 1. Nuages
Nocturnes : 1. Nuages --> Consolation 